In [1]:
import numpy as np
import gmsh
factory = gmsh.model.occ
model = gmsh.model

In [4]:
surf_int_path = r'/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/C0002233_int_CORT_int_11_point_insertion.npy'
surf_ext_path = r'/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/C0002233_int_CORT_ext_11_point_insertion.npy'

surf_int = np.load(surf_int_path)
surf_ext = np.load(surf_ext_path)

In [5]:
gmsh.initialize()

def offset_tags(entities):
    """
    get all entities of the model and add offset to make unique tags
    """
    for tag in entities:
        newTag_s = (tag[0], tag[1] + 1000)
        gmsh.model.setTag(dim=tag[0], tag=tag[1], newTag=newTag_s[1])
    return None

def surfaces_gmsh(x1, y1, z1, x2, y2 ,z2):
        """
        https://gitlab.onelab.info/gmsh/gmsh/-/issues/456
        https://bbanerjee.github.io/ParSim/fem/meshing/gmsh/quadrlateral-meshing-with-gmsh/
        """
        gmsh.option.setNumber("General.Terminal", 1)

        points1 = []
        for i in range(0, len(x1)):
            point1 = factory.addPoint(x1[i], y1[i], z1)
            points1 = np.append(points1, point1)
        points1 = np.append(points1, points1[0])
        B1 = factory.addBSpline(range(1, len(points1)), degree=3)
        C1 = factory.addCurveLoop([B1])
        
        points2 = []
        for i in range(0, len(x2)):
            point2 = factory.addPoint(x2[i], y2[i], z2)
            points2 = np.append(points2, point2)
        points2 = np.append(points2, points2[0])
        B2 = factory.addBSpline(range(len(points1) + 1, len(points1) + len(points2)), degree=3)
        C2 = factory.addCurveLoop([B2])

def finalize():
    factory.synchronize()
    offset_tags(factory.getEntities())
    gmsh.write('/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/surf_11_point_insertion.geo_unrolled')
    # gmsh.fltk.run()
    gmsh.finalize()

surfaces_gmsh(surf_ext[:, 0], surf_ext[:, 1], 2, surf_int[:, 0], surf_int[:, 1], 2)
finalize()

Info    : Writing '/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/surf_11_point_insertion.geo_unrolled'...
Info    : Done writing '/home/simoneponcioni/Documents/01_PHD/03_Methods/Meshing/Meshing/99_testing_prototyping/testing_transfinite_2D/surf_11_point_insertion.geo_unrolled'


In [ ]:
# Contributed by Roberto Agromayor

import gmsh
import sys
import numpy as np

gmsh.initialize()

# Set a target mesh size
lc = 2e-2

# Define the south B-Spline curve
a = 0.25
P1  = gmsh.model.occ.addPoint(0.00, 0.00, 0.00, lc)
P2  = gmsh.model.occ.addPoint(0.33, 0.00 + a, 0.00 + a, lc)
P3  = gmsh.model.occ.addPoint(0.66, 0.00 - a, 0.00 + a, lc)
P4  = gmsh.model.occ.addPoint(1.00, 0.00, 0.00, lc)
C1 = gmsh.model.occ.addBSpline([P1, P2, P3, P4], degree=3)

# Define the north B-Spline curve
P5  = gmsh.model.occ.addPoint(0.00, 1.00, 0.00, lc)
P6  = gmsh.model.occ.addPoint(0.33, 1.00 - a, 0.00 - a, lc)
P7  = gmsh.model.occ.addPoint(0.66, 1.00 + a, 0.00 - a, lc)
P8  = gmsh.model.occ.addPoint(1.00, 1.00, 0.00, lc)
C2 = gmsh.model.occ.addBSpline([P5, P6, P7, P8], degree=3)

# Define the east B-Spline curve
P9 = gmsh.model.occ.addPoint(0.00-a, 0.50, 0.00 + a, lc)
C3 = gmsh.model.occ.addBSpline([P1,P9, P5], degree=2)

# Define the west B-Spline curve
P10 = gmsh.model.occ.addPoint(1.00+a, 0.50, 0.00 - a, lc)
C4 = gmsh.model.occ.addBSpline([P4, P10, P8], degree=3)

# Create a BSpline surface filling the 4 curves:
W1 = gmsh.model.occ.addWire([C1, C3, C2, C4])

# gmsh.model.occ.addBSplineFilling(W1, type="Stretch")
gmsh.model.occ.addBSplineFilling(W1, type="Curved")
# gmsh.model.occ.addBSplineFilling(W1, type="Coons") # fails...

# Synchronize the CAD model
gmsh.model.occ.synchronize()

# Show the model
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

# Exit gmsh API
gmsh.finalize()